# Restaurant Reviews Analysis and Summarization

## Overview
This notebook processes and analyzes restaurant reviews using a Large Language Model (LLM) to generate concise summaries of customer feedback. The workflow is designed to handle Turkish restaurant reviews and create meaningful summaries that highlight key aspects of each establishment.

## Process Flow
1. **Data Preparation** (Cell 0)
   - Groups reviews by restaurant name
   - Combines reviews for each restaurant with comma separators
   - Saves the grouped data to "restoran_gruplu.csv"

2. **LLM Setup and Testing** (Cell 1)
   - Configures connection to a local LLM using LM Studio
   - Uses the "turkish-gemma-9b-v0.1-i1" model
   - Tests the model with a simple greeting prompt

3. **Review Summarization** (Cell 2)
   - Processes each restaurant's combined reviews through the LLM
   - Generates a concise paragraph summarizing key aspects:
     - Food quality and popular dishes
     - Service quality
     - Price points
     - Ambiance
     - Notable strengths and weaknesses
   - Includes error handling and rate limiting
   - Saves results to "restoran_yorumlar_ozetli.csv"

4. **Manual Processing** (Cell 3)
   - Handles 24 restaurants that exceeded the model's input token limit
   - Manually processes these cases using the same model and methodology


## Output
The final output is a CSV file containing:
- Restaurant names
- Combined reviews
- AI-generated summaries highlighting key aspects of each establishment

In [ ]:
import pandas as pd

# Excel dosyasını yükle
df = pd.read_excel("restoran_yorum.xlsx")

# A sütunu: Restoran adı, B sütunu: Yorumlar
# Restoran adına göre grupla, yorumları ',' ile birleştir
df_grouped = df.groupby("Restoran")["Yorum"].apply(
    lambda x: "','".join(x.dropna().astype(str))
).reset_index()

# Çıktıyı CSV olarak kaydet
df_grouped.to_csv("restoran_gruplu.csv", index=False, encoding="utf-8-sig")

print("Gruplama tamamlandı. Çıktı: restoran_gruplu.csv")

In [ ]:
from openai import OpenAI

# API endpoint'i LM Studio'nun açtığı localhost adresi
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

# Model ismi LM Studio’da listelenen model adı olmalı (örn: "TheBloke/Mistral-7B-Instruct")
response = client.chat.completions.create(
    model="turkish-gemma-9b-v0.1-i1",
    messages=[
        {"role": "system", "content": "Sana bir restoran için birleştirdiğim 'yorum', 'yorum' şeklinde birleştirilmiş tüm yorumları vereceğim. Bu yorumları bir arada değerlendirip restoranın öne çıkan özellikleri hakkında bir paragraflık özet çıkarmanı istiyorum."},
        {"role": "user", "content": "Merhaba, nasılsın?"}
    ]
)

print(response.choices[0].message.content)

Merhaba! İyiyim, teşekkür ederim. Siz nasılsınız? 😊

Yardımcı olmam gereken bir konunuz var mı?  😄


In [ ]:
import pandas as pd
from openai import OpenAI
import time

# 1) LM Studio API'ye bağlan
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

MODEL = "turkish-gemma-9b-v0.1-i1"   # LM Studio'da görünen model adı
INPUT_CSV = "restoran_gruplu.csv"  # giriş dosyası
OUTPUT_CSV = "restoran_yorumlar_ozetli.csv"  # çıkış dosyası

# 2) Veriyi yükle
df = pd.read_csv(INPUT_CSV)

# 3) Yorumları özetleme fonksiyonu
def summarize(text: str) -> str:
    text = "" if pd.isna(text) else str(text)
    if len(text) > 50000:  # Çok uzun metinleri kırp
        text = text[:50000]

    try:
        resp = client.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "system",
                    "content": (
                    """Sana bir restoran için birleştirdiğim 'yorum', 'yorum' şeklinde birleştirilmiş tüm yorumları vereceğim.
                    Bu yorumları bir arada değerlendirip restoranın öne çıkan özellikleri hakkında bir paragraflık özet çıkarmanı istiyorum."""
                    ),
                },
                {"role": "user", "content": text},
            ],
            temperature=0.3,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Hata: {e}"

# 4) Her satır için sırayla özet çıkar ve print et
ozetler = []
for i, yorum in enumerate(df["Yorum"].fillna(""), start=1):
    ozet = summarize(yorum)
    ozetler.append(ozet)
    print(f"{i}. satır özeti:\n{ozet}\n{'-'*60}")
    time.sleep(0.2)  # isteğe bağlı: API'yi yormamak için küçük bekleme

df["Özet"] = ozetler

# 5) Çıktıyı kaydet
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"Bitti ✅ Çıktı dosyası: {OUTPUT_CSV}")

1. satır özeti:
Restoran, özellikle Adana dürüm ve kebaplarıyla öne çıkıyor. Müşteriler lezzetli ve uygun fiyatlı yemekleri övgüyle anlatıyorlar. Bazı yorumlarda porsiyonların küçük olduğu belirtilse de genel olarak doyurucu bulunduğu vurgulanıyor. Çalışanların güler yüzlü ve samimi davranışları da sıkça övülüyor. Mekanın temizliği konusunda ise bazı eleştiriler mevcut, ancak hijyen konusuna daha fazla önem verilmesi durumunda restoranın deneyimini daha da olumlu hale getirebileceği düşünülüyor. Ayrıca çiğköfte ikramı gibi küçük detaylar müşteri memnuniyetini artırıyor.  

Özetle, lezzetli yemekleri, uygun fiyatları ve samimi hizmetiyle Etimesgut bölgesinde tercih edilen bir mekan olarak öne çıkıyor.
------------------------------------------------------------
2. satır özeti:
Bu restoran, özellikle **lezzetli Adana kebabı** ile öne çıkıyor. Müşteriler kaburga ve çöp şişi de övgüyle anarken, bazıları mezelerin çeşit ve lezzet açısından sınırlı olduğunu belirtiyor. 

Restoranın **hizmet 

Modelin input token sayısı aşıldığı için özet çıkarılamayan 24 restoran için tek tek manuel bir şekilde yine aynı modele üzerinden özetler çıkarılmıştır.